In [3]:
pip install xgboost tensorflow scikit-learn pandas numpy

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.0 MB 6.7 MB/s eta 0:00:11
   - -------------------------------------- 2.4/72.0 MB 7.5 MB/s eta 0:00:10
   -- ------------------------------------- 3.7/72.0 MB 7.3 MB/s eta 0:00:10
   --- ------------------------------------ 5.5/72.0 MB 7.3 MB/s eta 0:00:10
   --- ------------------------------------ 6.8/72.0 MB 7.2 MB/s eta 0:00:10
   ---- ----------------------------------- 8.4/72.0 MB 7.2 MB/s eta 0:00:09
   ----- ---------------------------------- 9.7/72.0 MB 7.2 MB/s eta 0:00:09
   ------ --------------------------------- 11.3/72.0 MB 7.2 MB/s eta 0:00:09
   ------- -------------------------------- 12.8/72.0 MB 7.1 MB/s eta 0:00:09
   -------- ------------------------------- 14.4/72.0 MB 7.2 MB/s eta 0:00:09
   -------- ------------------------------- 16.0/72.0 MB 7.2 MB/s eta 0:00:08
   --------- ------------------------------ 17.6/72.0 MB 7.2 MB/s eta 0:00:08
 

In [7]:
pip install torch torchvision torchaudio xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import warnings

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error
)

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

In [7]:
# ==================================================
# LOAD DATA
# ==================================================
FILE_PATH = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\39 BUS SYSTEM_FAULT_CONVERTED.xlsx"
df = pd.read_excel(FILE_PATH)

# ==================================================
# BASE FEATURES
# ==================================================
BASE_FEATURES = [
    "Phase Voltage A/Terminal in kV",
    "Phase Voltage B/Terminal in kV",
    "Phase Voltage C/Terminal in kV",
    "Phase Current A/Terminal in kA",
    "Phase Current B/Terminal in kA",
    "Phase Current C/Terminal in kA",
    "Terminal/Bus"
]

LABELS = ["Primary", "Fault Location", "Fault Line", "CCT"]

In [9]:
# ==================================================
# DATA CLEANING
# ==================================================
df = df[BASE_FEATURES + LABELS].dropna()
df = pd.get_dummies(df, columns=["Terminal/Bus"], drop_first=True)

FEATURES = [c for c in df.columns if c not in LABELS]

# ==================================================
# FEATURE SCALING
# ==================================================
scaler = StandardScaler()
X = scaler.fit_transform(df[FEATURES])

print("\n🤖 AUTOMATIC MODEL SELECTION: GRU vs XGBOOST vs ENSEMBLE")
print("======================================================")

# ==================================================
# PYTORCH GRU MODEL
# ==================================================
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, h = self.gru(x)
        out = self.fc(h[-1])
        return out


def train_gru(X_train, y_train, X_test, num_classes, epochs=40):
    X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_test  = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

    model = GRUModel(X_train.shape[2], 64, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for _ in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        test_outputs = model(X_test)
        probs = torch.softmax(test_outputs, dim=1).numpy()
        preds = np.argmax(probs, axis=1)

    return preds, probs


final_summary = {}

# ==================================================
# LOOP FOR EACH LABEL
# ==================================================
for label in LABELS:

    print("\n-----------------------------------------------")
    print(f"LABEL: {label}")
    print("-----------------------------------------------")

    y = LabelEncoder().fit_transform(df[label])
    n_classes = len(np.unique(y))

    if n_classes < 2:
        print("❌ Only one class present — skipped")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )

    # ==================================================
    # GRU MODEL
    # ==================================================
    gru_pred, gru_prob = train_gru(X_train, y_train, X_test, n_classes)
    acc_gru = accuracy_score(y_test, gru_pred) * 100

    # ==================================================
    # XGBOOST MODEL
    # ==================================================
    xgb = XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="multi:softprob",
        num_class=n_classes,
        eval_metric="mlogloss",
        random_state=42
    )

    xgb.fit(X_train, y_train)
    xgb_prob = xgb.predict_proba(X_test)
    xgb_pred = np.argmax(xgb_prob, axis=1)
    acc_xgb = accuracy_score(y_test, xgb_pred) * 100

    # ==================================================
    # ENSEMBLE (AUTO α SEARCH)
    # ==================================================
    best_alpha, best_ens_acc = None, 0
    best_ens_prob, best_ens_pred = None, None

    for alpha in np.arange(0.1, 1.0, 0.1):
        ens_prob = alpha * gru_prob + (1 - alpha) * xgb_prob
        ens_pred = np.argmax(ens_prob, axis=1)
        acc = accuracy_score(y_test, ens_pred) * 100

        if acc > best_ens_acc:
            best_ens_acc = acc
            best_alpha = alpha
            best_ens_prob = ens_prob
            best_ens_pred = ens_pred

    # ==================================================
    # MODEL SELECTION
    # ==================================================
    scores = {
        "GRU": acc_gru,
        "XGBoost": acc_xgb,
        "Ensemble": best_ens_acc
    }

    best_model = max(scores, key=scores.get)

    if best_model == "GRU":
        y_pred, y_prob = gru_pred, gru_prob
    elif best_model == "XGBoost":
        y_pred, y_prob = xgb_pred, xgb_prob
    else:
        y_pred, y_prob = best_ens_pred, best_ens_prob

    # ==================================================
    # METRICS
    # ==================================================
    acc = accuracy_score(y_test, y_pred) * 100
    prec = precision_score(y_test, y_pred, average="weighted") * 100
    rec = recall_score(y_test, y_pred, average="weighted") * 100
    f1 = f1_score(y_test, y_pred, average="weighted") * 100
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    try:
        roc_auc = roc_auc_score(y_test, y_prob, multi_class="ovr") * 100
    except:
        roc_auc = None

    # ==================================================
    # PRINT RESULTS
    # ==================================================
    print(f"GRU Accuracy        : {acc_gru:.2f}%")
    print(f"XGBoost Accuracy    : {acc_xgb:.2f}%")
    print(f"Ensemble Accuracy   : {best_ens_acc:.2f}% (alpha={best_alpha})")
    print(f"✅ SELECTED MODEL   : {best_model}")

    print("\n--- FINAL METRICS (Selected Model) ---")
    print(f"Accuracy (%)   : {acc:.2f}")
    print(f"Precision (%)  : {prec:.2f}")
    print(f"Recall (%)     : {rec:.2f}")
    print(f"F1-score (%)   : {f1:.2f}")
    print(f"ROC-AUC (%)    : {roc_auc:.2f}" if roc_auc else "ROC-AUC : Not defined")
    print(f"MAE            : {mae:.4f}")
    print(f"MSE            : {mse:.4f}")
    print(f"RMSE           : {rmse:.4f}")

    final_summary[label] = {
        "Best Model": best_model,
        "GRU Acc": acc_gru,
        "XGB Acc": acc_xgb,
        "ENS Acc": best_ens_acc
    }

# ==================================================
# FINAL SUMMARY
# ==================================================
print("\n🎯 FINAL MODEL SELECTION SUMMARY")
print("================================")
for lbl, res in final_summary.items():
    print(
        f"{lbl:15s} → {res['Best Model']:10s} | "
        f"GRU={res['GRU Acc']:.2f}% | "
        f"XGB={res['XGB Acc']:.2f}% | "
        f"ENS={res['ENS Acc']:.2f}%"
    )

print("\n✅ GRU + XGBOOST AUTOMATIC SELECTION COMPLETED")


🤖 AUTOMATIC MODEL SELECTION: GRU vs XGBOOST vs ENSEMBLE

-----------------------------------------------
LABEL: Primary
-----------------------------------------------
GRU Accuracy        : 98.61%
XGBoost Accuracy    : 100.00%
Ensemble Accuracy   : 100.00% (alpha=0.1)
✅ SELECTED MODEL   : XGBoost

--- FINAL METRICS (Selected Model) ---
Accuracy (%)   : 100.00
Precision (%)  : 100.00
Recall (%)     : 100.00
F1-score (%)   : 100.00
ROC-AUC (%)    : 100.00
MAE            : 0.0000
MSE            : 0.0000
RMSE           : 0.0000

-----------------------------------------------
LABEL: Fault Location
-----------------------------------------------
GRU Accuracy        : 58.66%
XGBoost Accuracy    : 92.84%
Ensemble Accuracy   : 92.77% (alpha=0.1)
✅ SELECTED MODEL   : XGBoost

--- FINAL METRICS (Selected Model) ---
Accuracy (%)   : 92.84
Precision (%)  : 92.85
Recall (%)     : 92.84
F1-score (%)   : 92.84
ROC-AUC : Not defined
MAE            : 0.0716
MSE            : 0.0716
RMSE           : 0.2